In [ ]:
# 패키지 import 구간
import math
import pandas as pd
from google.colab import drive

# Geocoding상호작용 패키지
import json
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import requests
import datetime
import math

from google.colab import drive

from geopy.geocoders import Nominatim

# 정규화 패키지
from sklearn.preprocessing import MinMaxScaler

# 드라이브 마운트, google maps api
drive.mount('/content/drive')
google_maps_API = ""

Mounted at /content/drive


In [ ]:
### 함수 정의 구간
# 해당 함수는 사용자로부터 input을 받고, 받은 input으로부터 위도, 경도를 추출하는 코드


# Google Map api 위도, 경도 추출 함수
def get_lat_lng(address, api_key):
    # 쿼리
    endpoint = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    # 요청 수령 및 json포맷으로 변환
    response = requests.get(endpoint)
    data = response.json()
    if data['status'] == 'OK':
        lat = data['results'][0]['geometry']['location']['lat']
        lng = data['results'][0]['geometry']['location']['lng']
        return lat, lng
    else:
        return None, None


# 지번 주소를 기반으로 위도 경도 추출
def getLocList(addlist):
  latitude = []
  longitude = []

  for i in addlist:
    lat, lng = get_lat_lng(i, google_maps_API)
    latitude.append(lat)
    longitude.append(lng)

  loc = pd.DataFrame({'위도' : latitude, '경도' : longitude})

  return loc

In [ ]:
### 함수 정의 구간
# 아래 함수는 사용자의 위도, 경도가 추출된 사용자의 input, 도시명을 받아와서 입지정보로 확장하는 함수

# 데이터 업로드 _utf
def upload_data_utf(file_name):
  rtn = pd.read_csv(file_name, encoding = 'utf-8')
  rtn = pd.DataFrame(rtn)

  return rtn

# 데이터 업로드 _cp949
def upload_data_cp(file_name):
  rtn = pd.read_csv(file_name, encoding = 'cp949')
  rtn = pd.DataFrame(rtn)

  return rtn

# 데이터 업로드 euc-kr
def upload_data_euc(file_name):
  rtn = pd.read_csv(file_name, encoding = 'euc-kr')
  rtn = pd.DataFrame(rtn)

  return rtn

# haversine 공식 함수
def haversine_distance(lat1, lon1, lat2, lon2):
  # 위도와 경도를 라디안으로 변환
  lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

  # Haversine 공식 계산
  dlon = lon2 - lon1
  dlat = lat2 - lat1
  a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
  c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  # 지구 반지름 (킬로미터)
  radius = 6371
  # 두 지점 간의 거리 계산 (킬로미터)
  distance = radius * c

  return distance

# 아파트와 병원 간의 거리 계산 후 데이터프레임에 추가하는 함수
def calculate_distances(apartRow, medicalDf):
    distances = []
    for _, medicalRow in medicalDf.iterrows():
        distance = haversine_distance(apartRow['위도'], apartRow['경도'], medicalRow['위도'], medicalRow['경도'])
        distances.append(distance)
    return pd.Series(distances, index=medicalDf['이름'])


# 입력 지역에 따라 입지 정보 데이터 업로드 다르게
def upload_location_data(location):
  locationName = ""
  if location == "안양":
    locationName = "AnyangData"
  else:
    locationName = "SuwonData"

  # 의료시설 데이터
  pharmacyDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/의료시설/약국데이터.csv')
  clinicDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/의료시설/의원데이터.csv')
  hospitalDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/의료시설/병원데이터.csv')
  generalHDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/의료시설/종합병원데이터.csv')

  # 상업시설 데이터
  commerceDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/상업시설/상업시설데이터.csv')

  # 편의시설 데이터
  parkDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/편의시설/공원데이터.csv')
  libraryDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/편의시설/도서관데이터.csv')

  # 교육시설 데이터
  kinderDf=  upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교육시설/유치원데이터.csv')
  lowSchoolDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교육시설/초등학교데이터.csv')
  middleSchoolDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교육시설/중학교데이터.csv')
  highSchoolDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교육시설/고등학교데이터.csv')
  academyDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교육시설/학원데이터.csv')

  # 교통시설 데이터
  subwayDf = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/' + locationName + '/교통시설/역사데이터.csv')

  return (pharmacyDf, clinicDf, hospitalDf, generalHDf, commerceDf, parkDf, libraryDf, kinderDf, lowSchoolDf, middleSchoolDf, highSchoolDf, academyDf, subwayDf)


def get_distance(ApartDF, cityName):
  # 1개의 아파트와 n개의 입지시설 데이터를 매칭시키고 거리를 구함.
  # 구해진 거리 중 최소의 값만 추출할 것.
  # 의료시설
  pharmacyDf, clinicDf, hospitalDf, generalHDf, commerceDf, parkDf, libraryDf, kinderDf, lowSchoolDf, middleSchoolDf, highSchoolDf, academyDf, subwayDf = upload_location_data(cityName)

  pharmacy_dt = ApartDF.apply(lambda row: calculate_distances(row, pharmacyDf), axis=1)
  pharmacy_minDt = pharmacy_dt.apply(lambda row : row.min(), axis = 1)

  clinic_dt = ApartDF.apply(lambda row: calculate_distances(row, clinicDf), axis=1)
  clinic_minDt = clinic_dt.apply(lambda row : row.min(), axis = 1)

  hospital_dt = ApartDF.apply(lambda row: calculate_distances(row, hospitalDf), axis=1)
  hospital_minDt = hospital_dt.apply(lambda row : row.min(), axis = 1)

  generalH_dt = ApartDF.apply(lambda row: calculate_distances(row, generalHDf), axis=1)
  generalH_minDt = generalH_dt.apply(lambda row : row.min(), axis = 1)

  # 상업시설
  commerce_dt = ApartDF.apply(lambda row: calculate_distances(row, commerceDf), axis=1)
  commerce_minDt = commerce_dt.apply(lambda row : row.min(), axis = 1)

  # 편의시설
  park_dt = ApartDF.apply(lambda row: calculate_distances(row, parkDf), axis=1)
  park_minDt = park_dt.apply(lambda row : row.min(), axis = 1)

  library_dt = ApartDF.apply(lambda row: calculate_distances(row, libraryDf), axis=1)
  library_minDt = library_dt.apply(lambda row : row.min(), axis = 1)

  # 교육시설
  kinder_dt = ApartDF.apply(lambda row: calculate_distances(row, kinderDf), axis=1)
  kinder_minDt = kinder_dt.apply(lambda row : row.min(), axis = 1)

  lowSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, lowSchoolDf), axis=1)
  lowSchool_minDt = lowSchool_dt.apply(lambda row : row.min(), axis = 1)

  middleSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, middleSchoolDf), axis=1)
  middleSchool_minDt = middleSchool_dt.apply(lambda row : row.min(), axis = 1)

  highSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, highSchoolDf), axis=1)
  highSchool_minDt = highSchool_dt.apply(lambda row : row.min(), axis = 1)

  academy_dt = ApartDF.apply(lambda row: calculate_distances(row, academyDf), axis=1)
  academy_minDt = academy_dt.apply(lambda row : row.min(), axis = 1)

  # 교통시설
  subway_dt = ApartDF.apply(lambda row: calculate_distances(row, subwayDf), axis=1)
  subway_minDt = subway_dt.apply(lambda row : row.min(), axis = 1)

  # 모든 아파트로부터 입지 간 최소 거리 데이터 하나로 합침
  LocationDF = pd.concat(
    [pharmacy_minDt, clinic_minDt, hospital_minDt, generalH_minDt, commerce_minDt, park_minDt, library_minDt, kinder_minDt, lowSchool_minDt, middleSchool_minDt, highSchool_minDt, academy_minDt, subway_minDt],
    axis=1,
    keys=['약국', '의원', '병원', '대병원', '대형상권', '공원', '도서관', '유치원', '초등학교', '중학교', '고등학교', '학원', '지하철']
    )

  return LocationDF


# 특정 거리 이내 입지 개수 구하기
def get_cnt(ApartDF, cityName, threshhold):
  # 의료시설
  pharmacyDf, clinicDf, hospitalDf, generalHDf, commerceDf, parkDf, libraryDf, kinderDf, lowSchoolDf, middleSchoolDf, highSchoolDf, academyDf, subwayDf = upload_location_data(cityName)

  pharmacy_dt = ApartDF.apply(lambda row: calculate_distances(row, pharmacyDf), axis=1)
  pharmacy_cnt = pharmacy_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  clinic_dt = ApartDF.apply(lambda row: calculate_distances(row, clinicDf), axis=1)
  clinic_cnt = clinic_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  hospital_dt = ApartDF.apply(lambda row: calculate_distances(row, hospitalDf), axis=1)
  hospital_cnt = hospital_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  generalH_dt = ApartDF.apply(lambda row: calculate_distances(row, generalHDf), axis=1)
  generalH_cnt = generalH_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  # 상업시설
  commerce_dt = ApartDF.apply(lambda row: calculate_distances(row, commerceDf), axis=1)
  commerce_cnt = commerce_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  # 편의시설
  park_dt = ApartDF.apply(lambda row: calculate_distances(row, parkDf), axis=1)
  park_cnt = park_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  library_dt = ApartDF.apply(lambda row: calculate_distances(row, libraryDf), axis=1)
  library_cnt = library_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  # 교육시설
  kinder_dt = ApartDF.apply(lambda row: calculate_distances(row, kinderDf), axis=1)
  kinder_cnt = kinder_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  lowSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, lowSchoolDf), axis=1)
  lowSchool_cnt = lowSchool_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  middleSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, middleSchoolDf), axis=1)
  middleSchool_cnt = middleSchool_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  highSchool_dt = ApartDF.apply(lambda row: calculate_distances(row, highSchoolDf), axis=1)
  highSchool_cnt = highSchool_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  academy_dt = ApartDF.apply(lambda row: calculate_distances(row, academyDf), axis=1)
  academy_cnt = academy_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  # 교통시설
  subway_dt = ApartDF.apply(lambda row: calculate_distances(row, subwayDf), axis=1)
  subway_cnt = subway_dt.apply(lambda row : (row <= threshhold).sum(), axis = 1)

  # 모든 아파트로부터 입지 간 최소 거리 데이터 하나로 합침
  LocationDF = pd.concat(
    [pharmacy_cnt, clinic_cnt, hospital_cnt, generalH_cnt, commerce_cnt, park_cnt, library_cnt, kinder_cnt, lowSchool_cnt, middleSchool_cnt, highSchool_cnt, academy_cnt, subway_cnt],
    axis=1,
    keys=['약국 수', '의원 수', '병원 수', '대병원 수', '대형상권 수', '공원 수', '도서관 수', '유치원 수', '초등학교 수', '중학교 수', '고등학교 수', '학원 수', '지하철 수']
    )

  return LocationDF

In [ ]:
# 사용자로부터 inputData를 받고 이를 위치정보까지 갖는 데이터프레임으로 확장할거임.
def get_expandedInputData(inputDF, cityName):
  # 입지종류별 최소거리 측정
  infoLocation_min = get_distance(inputDF, cityName)
  # 최소거리를 0.7로 설정하고 입지 개수 cnt
  infoLocation_cnt = get_cnt(inputDF, cityName, 0.7)

  inputData = pd.concat([inputDF, infoLocation_min, infoLocation_cnt], axis = 1)

  return inputData


# 사용자로부터 받은 input을 모델 학습용 데이터로 전환
def input_toDataFrame(year, dong, size, jibeon, cityName, guName):
  input_data = pd.DataFrame.from_records([{"건축년도" : year, "법정동" : dong, "전용면적" : size, "지번" : jibeon}])

  inputAdd = "경기 " + cityName + "시 " + guName + " " + dong + " " + input_data['지번']
  inputLocation = getLocList(inputAdd)

  inputLocation = pd.merge(input_data, inputLocation, left_index=True, right_index=True, how='inner')
  input_data = pd.merge(inputLocation, pd.DataFrame(inputAdd), left_index=True, right_index=True, how='inner')
  input_data = input_data.rename(columns={0 : "지번주소"})

  # inputData + 입지정보 데이터
  totalInputData = get_expandedInputData(input_data, cityName)

  inputCol = ['건축년도', '전용면적', '위도','경도',
            '약국', '의원','병원', '대병원',
            '대형상권','공원', '도서관',
            '유치원', '초등학교', '중학교', '고등학교', '학원',
            '지하철',
            '약국 수', '의원 수', '병원 수', '대병원 수',
            '대형상권 수','공원 수','도서관 수',
            '유치원 수', '초등학교 수', '중학교 수', '고등학교 수', '학원 수',
            '지하철 수']

  totalInputData = totalInputData[inputCol]
  totalInputData = scaling(totalInputData, inputCol, cityName)

  return totalInputData

In [ ]:
def scaling(data, scaleColumn, cityName):
  if cityName == "안양":
    minData = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/anyangMin.csv')
    maxData = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/AnyangData/anyangMax.csv')
  else:
    minData = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/SuwonData/suwonMin.csv')
    maxData = upload_data_utf('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/SuwonData/suwonMax.csv')

  rtn = data

  # 각 열에 대하여 데이터 정규화
  for col in rtn.columns:
    rtn[col] = min_max_scaling(rtn[col], minData[col], maxData[col])

  return rtn

def min_max_scaling(col, min_val, max_val):
  return (col - min_val) / (max_val - min_val)

# 모델 API

In [ ]:
# 모델 로드 (랜덤포레스트)
import joblib

# 모델 로드
rfModel = joblib.load('/content/drive/MyDrive/Colab Notebooks/헤도닉_집값예측/model_file.pkl')
%cd drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install flask-ngrok
!pip install flask==2.0.3
!pip install pyngrok==4.1.1
!pip install Werkzeug==2.2.2
!ngrok authtoken '' # input your ngrok token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok

In [ ]:
app = Flask(__name__, template_folder='templates')  # 절대 경로를 사용한 템플릿을 집어넣을거임.
run_with_ngrok(app)  # Initialize ngrok when the app is run

@app.route('/')
def predict():
    # 여기서 모델을 사용하여 예측하고 결과를 prediction 변수에 저장
    prediction = "Model's Prediction"

    return render_template('ModelTemplate.html')


@app.route('/submit', methods=['POST'])
def submit():
    year = int(request.form['year'])
    dong = str(request.form['dong'])
    size = float(request.form['size'])
    jibeon = str(request.form['jibeon'])
    cityName = str(request.form['cityName'])
    guName = str(request.form['guName'])

    print(year, dong, size, jibeon, cityName, guName)
    inputUsers = input_toDataFrame(year, dong, size, jibeon, cityName, guName)

    return render_template('output.html')

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5cea-34-85-166-242.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [25/Nov/2023 16:12:10] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Nov/2023 16:12:10] "GET /favicon.ico HTTP/1.1" 404 -
ERROR:__main__:Exception on /submit [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1518, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1516, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.10/dist-packages/flask/app.py", line 1502, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "<ipython-input-8-eb0d04e221c0>", line 27, in submit
    inputUsers = input_toDataFrame(year, dong, size, jibeon, cityName, guName)
  File "<ipython-input-4-8d6f9b0d0bcf>", line 15,

2003 비산동 84.918 425 안양 동안구
1
